In [87]:
import numpy as np 
import pandas as pd


# Data Preprocessing
data = pd.read_table('../dataset/u1base.txt', delim_whitespace=True, header = None, parse_dates=False)
dataset=data.values[:,:2]
feedback=data.values[:,2]
users=data.values[:,0]
observed_data=dataset[feedback>=3]
d=10
#Data Initialization
Users=np.random.normal(0,1,(users[-1],d))
Items=np.random.normal(0,1,(max(dataset[:,1]),d))
Bias=np.random.normal(0,1,(max(dataset[:,1])))

P_users=[]
A_users=[]

total_items=np.array(range(1,max(dataset[:,1])+1))
for i in range(dataset[-1,0]):
    P_users.append(dataset[users==i+1,1])
    A_users.append(np.array([item for item in list(total_items) if item not in list(P_users[i])]))

T=10
learning_rate=0.1
alpha_u=alpha_v=beta_v=0.1
        
perm=np.random.permutation(Users.shape[0])
Users=Users[perm]

In [96]:
def sigmoid_deriv(x):
    sigm = 1. / (1. + np.exp(-x))
    return sigm * (1. - sigm)


def sigmoid(x):
    sigm = 1. / (1. + np.exp(-x))
    return sigm

def preference(user,P,A):
    rp=0
    for i in range(P.shape[0]):
        rp+=np.dot(Users[user],Items[P[i]-1])+Bias[P[i]-1]
        
    ra=0
    for i in range(A.shape[0]):
        ra+=np.dot(Users[user],Items[A[i]-1])+Bias[A[i]-1]
    
    return rp/P.shape[0]-ra/A.shape[0]

def calc_mean(d,P,A):
    P_mean=np.zeros((d))
    for i in P:
        P_mean+=Items[i-1]
    A_mean=np.zeros((d))
    for i in A:
        A_mean+=Items[i-1]
    P_mean=P_mean/P.shape[0]
    A_mean=A_mean/A.shape[0]
    return P_mean,A_mean


    

In [109]:


for t1 in range(10):
    loss=0
    for t2 in range(Users.shape[0]):
        user=np.random.randint(users[-1])
        p_size=np.random.randint(d)+1
        a_size=np.random.randint(p_size)+1
        P_indices=np.random.randint(P_users[user].shape[0],size=p_size)
        P=P_users[user][P_indices]
        A_indices=np.random.randint(int(len(A_users[user])),size=a_size)
        A=A_users[user][A_indices]
        rpa=preference(user,P,A)
        #loss
        loss+=-np.log(sigmoid(rpa))
        #derivate of loss fn with respect to R_upa
        
        del_rpa=-sigmoid(-rpa)
        P_mean,A_mean=calc_mean(d,P,A)
        
        
        
        #update equation
        Users[user]-= learning_rate*((P_mean-A_mean)*del_rpa + alpha_u*Users[user])
        for i in P:
            Items[i-1]-= learning_rate*(del_rpa*Users[user]/P.shape[0] + alpha_v*Items[i-1])
            
        P_mean,A_mean=calc_mean(d,P,A)
        Users[user]-= learning_rate*((P_mean-A_mean)*del_rpa + alpha_u*Users[user])
        
        for i in A:
            Items[i-1]-= learning_rate*(-del_rpa*Users[user]/A.shape[0] + alpha_v*Items[i-1])
        
        P_mean,A_mean=calc_mean(d,P,A)
        Users[user]-= learning_rate*((P_mean-A_mean)*del_rpa + alpha_u*Users[user])
        
        for i in P:
            Bias[i-1]-= learning_rate*(del_rpa/P.shape[0] + beta_v*Bias[i-1] )
        for i in A:
            Bias[i-1]-= learning_rate*(del_rpa/A.shape[0] + beta_v*Bias[i-1] )
    loss_epoch=loss/t2

    print(loss_epoch)


    
        
        
        
        

0.3731179758378144
0.3636074732366792
0.3640144156974036
0.36569846864629346
0.36575362669665806
0.36156515604419914
0.37077232492515816
0.3687593893088698
0.36284769833019803
0.36785204615707084
0.36120564616467493
0.3635886056374764
0.37380930329172857
0.35054047826072565
0.36568699343073646
0.3549022720184879
0.3698941381812247
0.3647293043383903
0.3569446454192057
0.34763118743589255
0.37222282324837525
0.35803331961418117
0.3651771662258867
0.348013755851558
0.3582228623626535
0.3549471889340833
0.3636778027065858
0.36286994372743503
0.36186886802127255
0.3580643950057919
0.3572193929912349
0.36229761351711337
0.36482683524302334
0.35755915792088394
0.35186335832185883
0.360253210157724
0.35195758365712954
0.3573444015588237
0.3456322761171052
0.3490488922308171
0.34153158271623213
0.3475632136070984
0.35838929096932315
0.3567262685905391
0.363380698102749
0.35377817363186953
0.3549160674243371
0.3594671718854631
0.36546879665836496
0.3402692597940938
0.36239703666523665
0.3547687

KeyboardInterrupt: 